# Load Horizons Data into KeplerDB

In [1]:
# Core
import numpy as np
import pandas as pd

# Utility
import collections
import re
import glob
import time

# Database
import sqlalchemy
from sqlalchemy import create_engine

In [3]:
# MSE imports
import kepler_sieve
from horizons_files import hrzn_load_db, ast_elt_load_db, massive_body_load_db
from horizons_files import hrzn_txt2csv, hrzn_csv2df, hrzn_df2db, hrzn_txt2db, hrzn_csv2db
from asteroid_element import load_ast_elt, load_data_impl, convert_data
from astro_utils import anomaly_M2E, anomaly_E2f
import db_config

## Load all the Horizons CSV Files into JPL.HorizonsImport

In [ ]:
# hrzn_load()

## Load the Asteroid Orbital Elements from JPL

In [ ]:
# ast_elt_load_db()

## Testing - Load CSV, Display as DataFrame

In [ ]:
fname_txt = '../data/jpl/horizons/planets/daily/010_sun.txt'
# fname_txt = '../data/jpl/horizons/planets/daily/001_mercury_barycenter.txt'
# fname_txt = '../data/jpl/horizons/moons/daily/301_moon.txt'
# fname_txt = '../data/jpl/horizons/asteroids/weekly/ast_0001.txt'

In [ ]:
fname_csv = hrzn_txt2csv(fname_txt)

In [ ]:
fname_csv

In [ ]:
df = hrzn_csv2df(fname_csv)

In [ ]:
pd.set_option('display.max_rows', 10)

In [ ]:
df

## Testing - Populate Database from CSV

In [ ]:
connection_str = f'mysql+pymysql://{db_config.username}:{db_config.password}@{db_config.hostname}/JPL'

In [ ]:
engine = create_engine(connection_str)

In [ ]:
engine.table_names()

In [ ]:
# hrzn_txt2db(fname_txt)

# Masses

In [ ]:
fname = '../data/jpl/objects/masses.csv'

In [ ]:
df = pd.read_csv(fname)

In [ ]:
df.rename(columns = {'GM_AU3_per_Day2':'GM'}, inplace=True)

In [ ]:
sun_idx = df.loc[df.ParameterName == 'GMS'].index[0]

In [ ]:
df.GM.loc[sun_idx]

In [ ]:
gm_sun = df.GM.loc[9]
gm_sun

In [ ]:
param_to_body_name = {
    'GM1' : 'LB.Mercury Barycenter',
    'GM2' : 'LB.Venus Barycenter',
    'GM3' : 'LB.Earth',
    'GM4' : 'LB.Mars Barycenter',
    'GM5' : 'LB.Jupiter Barycenter',
    'GM6' : 'LB.Saturn Barycenter',
    'GM7' : 'LB.Uranus Barycenter',
    'GM8' : 'LB.Neptune Barycenter',
    'GM9' : 'LB.Pluto Barycenter',
    'GMS' : 'LB.Sun',
    'GMB' : 'LB.Earth Barycenter',
    'GMM' : 'LB.Moon',   
}

In [ ]:
import collections
param_map = collections.defaultdict(str)
for k, v in param_to_body_name.items():
    param_map[k] = v

In [ ]:
df['M'] = df['GM'] / gm_sun

In [ ]:
df['HorizonsBodyName'] = df.ParameterName.map(param_to_body_name)

In [ ]:
mask = df.HorizonsBodyName.isna()

In [ ]:
def param_name_to_ast_num(p):
    """Map parameter name to asteroid number"""
    return int(p[2:6])

In [ ]:
asteroid_num = df[mask].ParameterName.apply(param_name_to_ast_num)

In [ ]:
df['AsteroidNumber'] = asteroid_num

In [ ]:
df[0:20]

In [ ]:
with engine.connect() as conn:
    # Truncate the JPL.MassiveBodyImport database table
    sql = "truncate table JPL.MassiveBodyImport;"
    # Insert the data from the DataFrame to the database
    conn.execute(sql)
    df.to_sql(name='MassiveBodyImport', con=conn, schema='JPL', if_exists='append', index=False)

In [4]:
massive_body_load_db()